# 🎭 LLM-Based Deepfake Reasoning Agent

## Government Cyber-Fraud Prevention Project

### Why Deepfake Detection Matters

Deepfakes have become a critical threat in **digital arrest scams**, where fraudsters impersonate:
- Police officers
- Government officials
- Court judges
- Law enforcement agents

These AI-generated videos are used to:
1. **Intimidate victims** into believing they're under investigation
2. **Extort money** through fake legal threats
3. **Steal personal information** under the guise of official procedures

**This notebook represents a Stage-1 prototype** that uses LLM reasoning to identify suspicious videos before they can be used in scams.

## 🤖 What This Agent Is (Ed Donner Style)

### The Difference: Model-Based vs. Reasoning-Based Detection

**Traditional Model-Based Detection:**
- Requires thousands of labeled deepfake videos
- Needs CNNs, temporal models, face landmark tracking
- Takes weeks/months to train
- High accuracy but slow to deploy

**Our Reasoning-Based Agent (Stage-1):**
- Uses an LLM to **reason** about video characteristics
- No training required
- Can be deployed in hours
- Lower accuracy but provides **immediate value**

### Why Start with a Reasoning Agent?

> *"Start simple, prove value, then scale."* — Ed Donner

This agent:
1. **Validates the concept** quickly
2. **Provides early warnings** while we build better models
3. **Helps us understand** what features matter most
4. **Buys time** to collect training data

Think of it as a **smart triage system** — not perfect, but useful enough to catch obvious fakes and flag suspicious cases for human review.

## 🤖 Deepfake Detection Model

This notebook uses **Hugging Face's deepfake detection model** for real video analysis.

### Model: `prithivMLmods/deepfake-detector-model-v1`

**Features:**
- ✅ **No API key required** - runs locally on your machine
- ✅ **Free and open-source** - available on Hugging Face
- ✅ **Vision Transformer (ViT)** - state-of-the-art architecture
- ✅ **Binary classification** - Real vs Deepfake
- ✅ **Works offline** - after initial model download

### How It Works:

1. **Extract frames** from the video using OpenCV
2. **Analyze key frames** using the ViT model
3. **Aggregate results** across multiple frames
4. **Provide classification** with confidence score

### First Run:
The model (~400MB) will be downloaded automatically on first use. This may take a few minutes depending on your internet speed.

In [1]:
# Import necessary libraries
import os
import json
import numpy as np
from datetime import datetime
from PIL import Image

# For video processing
try:
    import cv2
    CV2_AVAILABLE = True
    print("✅ OpenCV available for video processing")
except ImportError:
    print("❌ OpenCV not installed. Install with: pip install opencv-python")
    CV2_AVAILABLE = False

# For deepfake detection model
try:
    from transformers import pipeline
    import torch
    HF_AVAILABLE = True
    print("✅ Transformers library available")
    
    # Check if GPU is available
    device = 0 if torch.cuda.is_available() else -1
    device_name = "GPU" if device == 0 else "CPU"
    print(f"✅ Using device: {device_name}")
except ImportError:
    print("❌ Transformers not installed. Install with: pip install transformers torch")
    HF_AVAILABLE = False

print(f"\n✅ Libraries imported successfully")
print(f"📅 Today's date: {datetime.now().strftime('%Y-%m-%d')}")

✅ OpenCV available for video processing


e:\github_projects\cg_police2\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Transformers library available
✅ Using device: CPU

✅ Libraries imported successfully
📅 Today's date: 2026-02-03


In [2]:
# Load the deepfake detection model
print("🔄 Loading deepfake detection model...")
print("   (First run will download ~400MB model - this may take a few minutes)\n")

if HF_AVAILABLE:
    try:
        # Load the model pipeline
        deepfake_classifier = pipeline(
            "image-classification",
            model="prithivMLmods/deepfake-detector-model-v1",
            device=device
        )
        
        print("✅ Model loaded successfully!")
        print(f"   Model: prithivMLmods/deepfake-detector-model-v1")
        print(f"   Device: {device_name}")
        MODEL_LOADED = True
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        print("   Make sure you have internet connection for first-time download.")
        MODEL_LOADED = False
else:
    print("❌ Cannot load model - transformers library not available")
    MODEL_LOADED = False

🔄 Loading deepfake detection model...
   (First run will download ~400MB model - this may take a few minutes)



e:\github_projects\cg_police2\venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--prithivMLmods--deepfake-detector-model-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 210/210 [00:00<00:00, 1107.63it/s, Materializing param=vision_

✅ Model loaded successfully!
   Model: prithivMLmods/deepfake-detector-model-v1
   Device: CPU


In [3]:
def extract_frames_from_video(video_path, num_frames=10):
    """
    Extract evenly spaced frames from a video for analysis.
    
    Args:
        video_path: Path to the video file
        num_frames: Number of frames to extract (default: 10)
    
    Returns:
        List of PIL Images
    """
    
    if not CV2_AVAILABLE:
        print("❌ OpenCV not available - cannot extract frames")
        return None
    
    if not os.path.exists(video_path):
        print(f"❌ Video file not found: {video_path}")
        return None
    
    try:
        video = cv2.VideoCapture(video_path)
        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        
        if total_frames == 0:
            print("❌ Could not read video file")
            return None
        
        # Calculate frame indices to extract
        frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
        
        frames = []
        for idx in frame_indices:
            video.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = video.read()
            
            if ret:
                # Convert BGR to RGB
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # Convert to PIL Image
                pil_image = Image.fromarray(frame_rgb)
                frames.append(pil_image)
        
        video.release()
        
        print(f"✅ Extracted {len(frames)} frames from video")
        return frames
    
    except Exception as e:
        print(f"❌ Error extracting frames: {e}")
        return None

print("📹 Frame extraction function ready")

📹 Frame extraction function ready


In [4]:
def analyze_video_for_deepfake(video_path, num_frames=10):
    """
    Analyze a video for deepfake content using Hugging Face model.
    
    Args:
        video_path: Path to the video file
        num_frames: Number of frames to analyze
    
    Returns:
        Dictionary with analysis results
    """
    
    if not MODEL_LOADED:
        return {
            'success': False,
            'error': 'Model not loaded. Please run the model loading cell first.'
        }
    
    # Extract frames
    print(f"\n🔍 Extracting {num_frames} frames from video...")
    frames = extract_frames_from_video(video_path, num_frames)
    
    if not frames:
        return {
            'success': False,
            'error': 'Failed to extract frames from video'
        }
    
    # Analyze each frame
    print(f"\n🤖 Analyzing frames with deepfake detection model...")
    
    frame_results = []
    deepfake_scores = []
    
    for i, frame in enumerate(frames):
        try:
            # Run classification
            result = deepfake_classifier(frame)
            
            # Parse result (format: [{'label': 'Fake', 'score': 0.95}, {'label': 'Real', 'score': 0.05}])
            fake_score = next((r['score'] for r in result if 'fake' in r['label'].lower()), 0)
            
            frame_results.append({
                'frame_index': i,
                'predictions': result,
                'deepfake_score': fake_score
            })
            
            deepfake_scores.append(fake_score)
            
            print(f"   Frame {i+1}/{num_frames}: {result[0]['label']} ({result[0]['score']:.2%})")
        
        except Exception as e:
            print(f"   ⚠️ Error analyzing frame {i}: {e}")
    
    # Aggregate results
    avg_deepfake_score = np.mean(deepfake_scores)
    is_deepfake = avg_deepfake_score > 0.5
    confidence = avg_deepfake_score if is_deepfake else (1 - avg_deepfake_score)
    
    return {
        'success': True,
        'is_deepfake': is_deepfake,
        'confidence': confidence,
        'avg_deepfake_score': avg_deepfake_score,
        'frames_analyzed': len(frames),
        'frame_results': frame_results
    }

print("🎯 Deepfake analysis function ready")

🎯 Deepfake analysis function ready


In [5]:
def print_deepfake_report(video_path, result):
    """
    Print a formatted deepfake analysis report.
    """
    print("\n" + "="*80)
    print("🎭 DEEPFAKE ANALYSIS REPORT")
    print("="*80)
    print(f"\n📹 Video: {os.path.basename(video_path)}")
    
    if not result['success']:
        print(f"\n❌ ANALYSIS FAILED")
        print(f"Error: {result['error']}")
        print("\n" + "="*80)
        return
    
    print(f"📊 Frames Analyzed: {result['frames_analyzed']}")
    print("\n" + "-"*80)
    
    classification = "DEEPFAKE" if result['is_deepfake'] else "REAL"
    confidence_pct = result['confidence'] * 100
    
    print(f"\n🎯 CLASSIFICATION: {classification}")
    print(f"📊 CONFIDENCE: {confidence_pct:.1f}%")
    print(f"📈 Average Deepfake Score: {result['avg_deepfake_score']:.2%}")
    
    # Provide reasoning
    print(f"\n💡 ANALYSIS:")
    if result['is_deepfake']:
        if confidence_pct > 80:
            print("   Strong indicators of deepfake manipulation detected across multiple frames.")
        elif confidence_pct > 60:
            print("   Moderate indicators of deepfake manipulation detected.")
        else:
            print("   Some indicators of manipulation detected, but confidence is low.")
    else:
        if confidence_pct > 80:
            print("   Video appears authentic with high confidence.")
        elif confidence_pct > 60:
            print("   Video appears authentic with moderate confidence.")
        else:
            print("   Video appears authentic, but confidence is low.")
    
    # Recommendations
    print(f"\n✅ RECOMMENDATION:")
    if result['is_deepfake'] and confidence_pct > 70:
        print("   ⚠️ HIGH RISK: Flag for immediate investigation.")
        print("   Do not trust the content. Report to authorities if used in scam.")
    elif result['is_deepfake'] and confidence_pct > 50:
        print("   ⚠️ MEDIUM RISK: Recommend human review and additional verification.")
        print("   Exercise caution and verify through alternative sources.")
    else:
        print("   ✅ LOW RISK: Video appears authentic.")
        print("   Still verify source and context if used for important decisions.")
    
    print("\n" + "="*80)

print("📄 Report display function ready")

📄 Report display function ready


## 🎬 Analyze Your Video

Provide the path to your video file below.

### Requirements:
- ✅ Video file must exist on your system
- ✅ Supported formats: MP4, AVI, MOV, MKV, etc.
- ✅ Model will analyze 10 frames (adjustable)
- ⚠️ First run will download the model (~400MB)

In [6]:
# ========================================
# USER INPUT: Specify your video file
# ========================================

# Path to your video file
VIDEO_PATH = "fake_officer_call.mp4"  # Change this to your video file path

# Number of frames to analyze (more frames = more accurate but slower)
NUM_FRAMES = 10

print("📋 Video Analysis Configuration:")
print(f"   Video Path: {VIDEO_PATH}")
print(f"   Frames to Analyze: {NUM_FRAMES}")
print("\n" + "="*80)

📋 Video Analysis Configuration:
   Video Path: fake_officer_call.mp4
   Frames to Analyze: 10



In [7]:
# ========================================
# RUN DEEPFAKE ANALYSIS
# ========================================

if not os.path.exists(VIDEO_PATH):
    print(f"\n❌ Video file not found: {VIDEO_PATH}")
    print("   Please check the path and try again.")
else:
    print(f"\n🚀 Starting deepfake analysis...")
    print(f"   This may take 30 seconds to 2 minutes depending on your hardware.\n")
    
    # Run analysis
    result = analyze_video_for_deepfake(VIDEO_PATH, NUM_FRAMES)
    
    # Display report
    print_deepfake_report(VIDEO_PATH, result)
    
    if result['success']:
        print("\n✅ Analysis complete!")
        print("\n⚠️ Note: Always verify with human review for critical decisions.")


🚀 Starting deepfake analysis...
   This may take 30 seconds to 2 minutes depending on your hardware.


🔍 Extracting 10 frames from video...
✅ Extracted 10 frames from video

🤖 Analyzing frames with deepfake detection model...
   Frame 1/10: Fake (54.81%)
   Frame 2/10: Real (53.82%)
   Frame 3/10: Real (50.42%)
   Frame 4/10: Fake (52.23%)
   Frame 5/10: Real (53.06%)
   Frame 6/10: Real (53.87%)
   Frame 7/10: Real (57.19%)
   Frame 8/10: Real (50.73%)
   Frame 9/10: Real (53.28%)
   Frame 10/10: Real (54.08%)

🎭 DEEPFAKE ANALYSIS REPORT

📹 Video: fake_officer_call.mp4
📊 Frames Analyzed: 10

--------------------------------------------------------------------------------

🎯 CLASSIFICATION: REAL
📊 CONFIDENCE: 51.9%
📈 Average Deepfake Score: 48.06%

💡 ANALYSIS:
   Video appears authentic, but confidence is low.

✅ RECOMMENDATION:
   ✅ LOW RISK: Video appears authentic.
   Still verify source and context if used for important decisions.


✅ Analysis complete!

⚠️ Note: Always verify wit

## ⚠️ IMPORTANT NOTES

### What This System Does:

This notebook uses a **Vision Transformer (ViT) model** trained on deepfake datasets:
- ✅ **Real frame-by-frame analysis** (not just metadata)
- ✅ **No API key required** - runs completely locally
- ✅ **Works offline** - after initial model download
- ✅ **Free and open-source**

### Limitations:

- ⚠️ **Image-based detection only** - analyzes frames, not audio-visual sync
- ⚠️ **Not 100% accurate** - sophisticated deepfakes may evade detection
- ⚠️ **Requires GPU for speed** - CPU inference is slower
- ⚠️ **Model size** - ~400MB download on first use
- ⚠️ **Frame sampling** - only analyzes subset of frames (not entire video)

### Accuracy:

The model has been trained on common deepfake datasets and works well for:
- ✅ Face-swap deepfakes
- ✅ GAN-generated faces
- ✅ Common deepfake tools (DeepFaceLab, FaceSwap, etc.)

May struggle with:
- ❌ Very high-quality deepfakes
- ❌ Novel deepfake techniques
- ❌ Audio deepfakes (voice cloning)

### Use Cases:

**✅ Good for:**
- Initial screening of suspicious videos
- Rapid triage in scam investigations
- Educational purposes
- Offline/air-gapped environments

**❌ NOT suitable for:**
- Forensic evidence without additional verification
- 100% certainty requirements
- Real-time video analysis

### Recommendations:

1. **Always verify results** with human analysts
2. **Use multiple detection methods** for critical cases
3. **Analyze more frames** (increase NUM_FRAMES) for better accuracy
4. **Check audio separately** - this model only analyzes visual content

---

**Model**: [prithivMLmods/deepfake-detector-model-v1](https://huggingface.co/prithivMLmods/deepfake-detector-model-v1)  
**License**: Open-source (check Hugging Face for details)  
**Version**: 2.0 (Local Model-based)